In [ ]:


import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Load dataset
df = pd.read_csv("dataset/OnlineRetail.csv", encoding="ISO-8859-1")

# Preview
print(df.head())

#  Data Cleaning 
df.dropna(subset=["CustomerID"], inplace=True)
df = df[df["Quantity"] > 0]
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

# --- Q2: Total Revenue ---
df["Revenue"] = df["Quantity"] * df["UnitPrice"]
total_revenue = df["Revenue"].sum()
print("Total Revenue:", total_revenue)

# --- Q6: Monthly Revenue Trend ---
monthly = df.groupby(df["InvoiceDate"].dt.to_period("M"))["Revenue"].sum()
monthly.plot(kind="line", figsize=(10,5), title="Monthly Revenue Trend")
plt.show()

# --- Q7: Customers who spent > £1000 ---
customer_spend = df.groupby("CustomerID")["Revenue"].sum()
high_value_customers = customer_spend[customer_spend > 1000]
print("High value customers:", high_value_customers.head())

# --- Q11: RFM Analysis ---
snapshot_date = df["InvoiceDate"].max() + pd.Timedelta(days=1)

rfm = df.groupby("CustomerID").agg({
    "InvoiceDate": lambda x: (snapshot_date - x.max()).days,
    "InvoiceNo": "count",
    "Revenue": "sum"
})
rfm.rename(columns={
    "InvoiceDate": "Recency",
    "InvoiceNo": "Frequency",
    "Revenue": "Monetary"
}, inplace=True)

print(rfm.head())

# --- Q14: Cohort Analysis ---
df["InvoiceMonth"] = df["InvoiceDate"].dt.to_period("M")
cohort = df.groupby("CustomerID")["InvoiceMonth"].min()
df["CohortMonth"] = df["CustomerID"].map(cohort)

cohort_data = df.groupby(["CohortMonth", "InvoiceMonth"])["CustomerID"].nunique().reset_index()
cohort_pivot = cohort_data.pivot(index="CohortMonth", columns="InvoiceMonth", values="CustomerID")
sns.heatmap(cohort_pivot, cmap="Blues", annot=True, fmt="g")
plt.title("Cohort Analysis - Active Customers")
plt.show()
